In [1]:
!nvcc --version
%pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-q5yv4rdo
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-q5yv4rdo
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=c1fff156b45bd4438ec9bfb8e07997d81144016243c4484292de0466f4d52a95
  Stored in directory: /tmp/pip-ephem-wheel-cache-l8egqg7z/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
!nvidia-smi

Fri Jan  5 14:01:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
%%writefile sha256.cuh
#ifndef SHA256_H
#define SHA256_H

#include <cstdint>

/****************************** MACROS ******************************/
#define SHA256_BLOCK_SIZE 32            // SHA256 outputs a 32-byte digest

#define ROTLEFT(a,b) (((a) << (b)) | ((a) >> (32-(b))))
#define ROTRIGHT(a,b) (((a) >> (b)) | ((a) << (32-(b))))

#define CH(x,y,z) (((x) & (y)) ^ (~(x) & (z)))
#define MAJ(x,y,z) (((x) & (y)) ^ ((x) & (z)) ^ ((y) & (z)))
#define EP0(x) (ROTRIGHT(x,2) ^ ROTRIGHT(x,13) ^ ROTRIGHT(x,22))
#define EP1(x) (ROTRIGHT(x,6) ^ ROTRIGHT(x,11) ^ ROTRIGHT(x,25))
#define SIG0(x) (ROTRIGHT(x,7) ^ ROTRIGHT(x,18) ^ ((x) >> 3))
#define SIG1(x) (ROTRIGHT(x,17) ^ ROTRIGHT(x,19) ^ ((x) >> 10))

/****************************** DATA TYPES ******************************/
typedef unsigned char BYTE;             // 8-bit byte
typedef uint32_t  WORD;                  // 32-bit word, change to "long" for 16-bit machines

typedef struct {
    BYTE data[64];
    WORD datalen;
    unsigned long long bitlen;
    WORD state[8];
} SHA256_CTX;

__constant__ WORD dev_k[64];

static const WORD host_k[64] = {
    0x428a2f98,0x71374491,0xb5c0fbcf,0xe9b5dba5,0x3956c25b,0x59f111f1,0x923f82a4,0xab1c5ed5,
    0xd807aa98,0x12835b01,0x243185be,0x550c7dc3,0x72be5d74,0x80deb1fe,0x9bdc06a7,0xc19bf174,
    0xe49b69c1,0xefbe4786,0x0fc19dc6,0x240ca1cc,0x2de92c6f,0x4a7484aa,0x5cb0a9dc,0x76f988da,
    0x983e5152,0xa831c66d,0xb00327c8,0xbf597fc7,0xc6e00bf3,0xd5a79147,0x06ca6351,0x14292967,
    0x27b70a85,0x2e1b2138,0x4d2c6dfc,0x53380d13,0x650a7354,0x766a0abb,0x81c2c92e,0x92722c85,
    0xa2bfe8a1,0xa81a664b,0xc24b8b70,0xc76c51a3,0xd192e819,0xd6990624,0xf40e3585,0x106aa070,
    0x19a4c116,0x1e376c08,0x2748774c,0x34b0bcb5,0x391c0cb3,0x4ed8aa4a,0x5b9cca4f,0x682e6ff3,
    0x748f82ee,0x78a5636f,0x84c87814,0x8cc70208,0x90befffa,0xa4506ceb,0xbef9a3f7,0xc67178f2
};

/*************************** FUNCTION DECLARATIONS ***************************/
char* print_sha(BYTE* buff);
__device__ void sha256_init(SHA256_CTX* ctx);
__device__ void sha256_update(SHA256_CTX* ctx, const BYTE data[], size_t len);
__device__ void sha256_final(SHA256_CTX* ctx, BYTE hash[]);

__device__ void mycpy12(uint32_t* d, const uint32_t* s) {
#pragma unroll 3
    for (int k = 0; k < 3; k++) d[k] = s[k];
}

__device__ void mycpy16(uint32_t* d, const uint32_t* s) {
#pragma unroll 4
    for (int k = 0; k < 4; k++) d[k] = s[k];
}

__device__ void mycpy32(uint32_t* d, const uint32_t* s) {
#pragma unroll 8
    for (int k = 0; k < 8; k++) d[k] = s[k];
}

__device__ void mycpy44(uint32_t* d, const uint32_t* s) {
#pragma unroll 11
    for (int k = 0; k < 11; k++) d[k] = s[k];
}

__device__ void mycpy48(uint32_t* d, const uint32_t* s) {
#pragma unroll 12
    for (int k = 0; k < 12; k++) d[k] = s[k];
}

__device__ void mycpy64(uint32_t* d, const uint32_t* s) {
#pragma unroll 16
    for (int k = 0; k < 16; k++) d[k] = s[k];
}

__device__ void sha256_transform(SHA256_CTX* ctx, const BYTE data[]) {
    WORD a, b, c, d, e, f, g, h, i, j, t1, t2, m[64];
    WORD S[8];

#pragma unroll 16
    for (i = 0, j = 0; i < 16; ++i, j += 4)
        m[i] = (data[j] << 24) | (data[j + 1] << 16) | (data[j + 2] << 8) | (data[j + 3]);

#pragma unroll 64
    for (; i < 64; ++i)
        m[i] = SIG1(m[i - 2]) + m[i - 7] + SIG0(m[i - 15]) + m[i - 16];

    a = ctx->state[0];
    b = ctx->state[1];
    c = ctx->state[2];
    d = ctx->state[3];
    e = ctx->state[4];
    f = ctx->state[5];
    g = ctx->state[6];
    h = ctx->state[7];

#pragma unroll 64
    for (i = 0; i < 64; ++i) {
        t1 = h + EP1(e) + CH(e, f, g) + dev_k[i] + m[i];
        t2 = EP0(a) + MAJ(a, b, c);
        h = g;
        g = f;
        f = e;
        e = d + t1;
        d = c;
        c = b;
        b = a;
        a = t1 + t2;
    }

    ctx->state[0] += a;
    ctx->state[1] += b;
    ctx->state[2] += c;
    ctx->state[3] += d;
    ctx->state[4] += e;
    ctx->state[5] += f;
    ctx->state[6] += g;
    ctx->state[7] += h;
}

__device__ void sha256_init(SHA256_CTX* ctx) {
    ctx->datalen = 0;
    ctx->bitlen = 0;
    ctx->state[0] = 0x6a09e667;
    ctx->state[1] = 0xbb67ae85;
    ctx->state[2] = 0x3c6ef372;
    ctx->state[3] = 0xa54ff53a;
    ctx->state[4] = 0x510e527f;
    ctx->state[5] = 0x9b05688c;
    ctx->state[6] = 0x1f83d9ab;
    ctx->state[7] = 0x5be0cd19;
}

__device__ void sha256_update(SHA256_CTX* ctx, const BYTE data[], size_t len) {
    WORD i;

    // for each byte in message
    for (i = 0; i < len; ++i) {
        // ctx->data == message 512 bit chunk
        ctx->data[ctx->datalen] = data[i];
        ctx->datalen++;
        if (ctx->datalen == 64) {
            sha256_transform(ctx, ctx->data);
            ctx->bitlen += 512;
            ctx->datalen = 0;
        }
    }
}

__device__ void sha256_final(SHA256_CTX* ctx, BYTE hash[]) {
    WORD i;

    i = ctx->datalen;

    // Pad whatever data is left in the buffer.
    if (ctx->datalen < 56) {
        ctx->data[i++] = 0x80;
        while (i < 56)
            ctx->data[i++] = 0x00;
    }
    else {
        ctx->data[i++] = 0x80;
        while (i < 64)
            ctx->data[i++] = 0x00;
        sha256_transform(ctx, ctx->data);
        memset(ctx->data, 0, 56);
    }

    // Append to the padding the total message's length in bits and transform.
    ctx->bitlen += ctx->datalen * 8;
    ctx->data[63] = ctx->bitlen;
    ctx->data[62] = ctx->bitlen >> 8;
    ctx->data[61] = ctx->bitlen >> 16;
    ctx->data[60] = ctx->bitlen >> 24;
    ctx->data[59] = ctx->bitlen >> 32;
    ctx->data[58] = ctx->bitlen >> 40;
    ctx->data[57] = ctx->bitlen >> 48;
    ctx->data[56] = ctx->bitlen >> 56;
    sha256_transform(ctx, ctx->data);

    // Since this implementation uses little endian byte ordering and SHA uses big endian,
    // reverse all the bytes when copying the final state to the output hash.
    for (i = 0; i < 4; ++i) {
        hash[i] = (ctx->state[0] >> (24 - i * 8)) & 0x000000ff;
        hash[i + 4] = (ctx->state[1] >> (24 - i * 8)) & 0x000000ff;
        hash[i + 8] = (ctx->state[2] >> (24 - i * 8)) & 0x000000ff;
        hash[i + 12] = (ctx->state[3] >> (24 - i * 8)) & 0x000000ff;
        hash[i + 16] = (ctx->state[4] >> (24 - i * 8)) & 0x000000ff;
        hash[i + 20] = (ctx->state[5] >> (24 - i * 8)) & 0x000000ff;
        hash[i + 24] = (ctx->state[6] >> (24 - i * 8)) & 0x000000ff;
        hash[i + 28] = (ctx->state[7] >> (24 - i * 8)) & 0x000000ff;
    }
}

#endif   // SHA256_H


Writing sha256.cuh


In [6]:
%%writefile password_crack_cuda1.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "sha256.cuh"

#define BLOCK_SIZE 16 // Define a block size for CUDA kernel
#define GRID_SIZE  16 // Define a grid size for CUDA kernel

__global__ void CudaCryptKernel(const char* rawPassword, char* encryptedPassword, int passwordLength) {
    int index = blockIdx.x * blockDim.x + threadIdx.x; // Calculate global index

    if (index < passwordLength) {
        SHA256_CTX ctx;
        sha256_init(&ctx);
        sha256_update(&ctx, (const BYTE*)&rawPassword[index * 4], 4);
        sha256_final(&ctx, (BYTE*)&encryptedPassword[index * 2 * SHA256_BLOCK_SIZE]);
    }
}

bool CheckPasswordPattern(const char* password) {
    // Check if the password matches the pattern "AA00" to "ZZ99"
    if (password[0] >= 'A' && password[0] <= 'Z' &&
        password[1] >= 'A' && password[1] <= 'Z' &&
        password[2] >= '0' && password[2] <= '9' &&
        password[3] >= '0' && password[3] <= '9') {
        return true;
    }
    return false;
}

int main() {
    char targetPassword[] = "KG67";
    char targetEncryptedPassword[2 * SHA256_BLOCK_SIZE + 1]; // For 32-character hexadecimal string + null terminator

    char* gpuTargetPassword;
    char* gpuTargetEncryptedPassword;

    cudaMalloc((void**)&gpuTargetPassword, sizeof(targetPassword));
    cudaMalloc((void**)&gpuTargetEncryptedPassword, 2 * SHA256_BLOCK_SIZE + 1);

    cudaMemcpy(gpuTargetPassword, targetPassword, sizeof(targetPassword), cudaMemcpyHostToDevice);

    if (!CheckPasswordPattern(targetPassword)) {
        printf("Error: Password should match the pattern 'AA00' to 'ZZ99'.\n");
        return 1; // Exit with an error code
    }

    dim3 blocks(GRID_SIZE);
    dim3 threads(BLOCK_SIZE);

    CudaCryptKernel<<<blocks, threads>>>(gpuTargetPassword, gpuTargetEncryptedPassword, 1); // 1 password to process
    cudaDeviceSynchronize();

    cudaMemcpy(targetEncryptedPassword, gpuTargetEncryptedPassword, 2 * SHA256_BLOCK_SIZE + 1, cudaMemcpyDeviceToHost);

    cudaFree(gpuTargetPassword);
    cudaFree(gpuTargetEncryptedPassword);

    printf("Target Password: %s\n", targetPassword);
    printf("Target Encrypted Password: ");
    for (int i = 0; i < SHA256_BLOCK_SIZE; i++) {
        printf("%02x", (unsigned char)targetEncryptedPassword[i]);
    }
    printf("\n");

    return 0;
}


Writing password_crack_cuda1.cu


In [7]:
%%shell
nvcc -o password_crack_cuda1 password_crack_cuda1.cu
./password_crack_cuda1

sha256.cuh(81): warning #177-D: variable "S" was declared but never referenced
      WORD S[8];
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Target Password: KG67
Target Encrypted Password: 992deb008af6fdfc8fa22f502c77f6e2edcebf5138cb446e61cc4fa10788af1f


In [8]:
%%writefile password_crack_cuda2.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "sha256.cuh"  // Include the header for your SHA-256 implementation

#define SHA256_BLOCK_SIZE 32
#define GRID_SIZE 32
#define BLOCK_SIZE 32
#define TOTAL_PASSWORDS (GRID_SIZE * BLOCK_SIZE)

__device__ __managed__ int found = 0;
__device__ __managed__ char correctPassword[5];

__global__ void CudaCryptKernel(BYTE* targetHash) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    char rawPassword[5];

    // Calculate the starting index for this thread block
    int startIdx = idx * TOTAL_PASSWORDS;

    for (int p = 0; p < TOTAL_PASSWORDS; p++) {
        // Generate a potential password based on the thread block's index
        rawPassword[0] = 'A' + ((startIdx + p) % 26); // First character: 'A' to 'Z'
        rawPassword[1] = 'A' + (((startIdx + p) / 26) % 26); // Second character: 'A' to 'Z'
        rawPassword[2] = '0' + (((startIdx + p) / (26 * 26)) % 10); // Third character: '0' to '9'
        rawPassword[3] = '0' + (((startIdx + p) / (26 * 26 * 10)) % 10); // Fourth character: '0' to '9'
        rawPassword[4] = '\0'; // Null-terminator for string

        BYTE outputHash[SHA256_BLOCK_SIZE];
        SHA256_CTX ctx;
        sha256_init(&ctx);
        sha256_update(&ctx, (const BYTE*)rawPassword, 4);
        sha256_final(&ctx, outputHash);

        // Check if the hashes match
        bool hashesMatch = true;
        for (int i = 0; i < SHA256_BLOCK_SIZE; i++) {
            if (outputHash[i] != targetHash[i]) {
                hashesMatch = false;
                break;
            }
        }

        if (hashesMatch) {
            memcpy(correctPassword, rawPassword, 4);
            found = 1;
            return; // Exit early if found
        }
    }
}

int main() {
    const char* targetHashString = "992deb008af6fdfc8fa22f502c77f6e2edcebf5138cb446e61cc4fa10788af1f";
    BYTE targetHash[SHA256_BLOCK_SIZE];

    // Check if the target hash string has exactly 64 characters
    if (strlen(targetHashString) != 64) {
        printf("Error: The target hash string must have exactly 64 characters.\n");
        return 1; // Exit with an error code
    }

    // Convert hash string to byte array
    for (int i = 0; i < SHA256_BLOCK_SIZE; i++) {
        targetHash[i] = (targetHashString[2 * i] >= 'a') ? (targetHashString[2 * i] - 'a' + 10) : (targetHashString[2 * i] - '0');
        targetHash[i] = (targetHash[i] << 4) | ((targetHashString[2 * i + 1] >= 'a') ? (targetHashString[2 * i + 1] - 'a' + 10) : (targetHashString[2 * i + 1] - '0'));
    }

    BYTE* gpuTargetHash;
    cudaMalloc(&gpuTargetHash, SHA256_BLOCK_SIZE);
    cudaMemcpy(gpuTargetHash, targetHash, SHA256_BLOCK_SIZE, cudaMemcpyHostToDevice);

    // Launch kernel with multiple thread blocks
    CudaCryptKernel<<<GRID_SIZE, BLOCK_SIZE>>>(gpuTargetHash);
    cudaDeviceSynchronize();

    if (found) {
        printf("Decrypted Password found: %s\n", correctPassword);
    } else {
        printf("Password not found.\n");
    }

    cudaFree(gpuTargetHash);
    return 0;
}



Writing password_crack_cuda2.cu


In [9]:
%%shell
nvcc -o password_crack_cuda2 password_crack_cuda2.cu
./password_crack_cuda2

sha256.cuh(81): warning #177-D: variable "S" was declared but never referenced
      WORD S[8];
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Decrypted Password found: KG67
